<a href="https://colab.research.google.com/github/nxlr/sentiment-intent-analyzer/blob/main/sentiment_intent_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Install Required Packages
!pip install transformers
!pip install git+https://github.com/tweepy/tweepy.git
!pip install tensorflow
!pip install streamlit
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/tweepy/tweepy.git to /tmp/pip-req-build-d2c9mfa_
  Running command git clone -q https://github.com/tweepy/tweepy.git /tmp/pip-req-build-d2c9mfa_
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import RobertaTokenizer
import tweepy
import streamlit as st
import sentencepiece as spm
from transformers import pipeline

In [ ]:
# Set API keys and Authentication method
api_key = 'ixHfMyHeS4kZoNrTzY4VWxwmV'
api_key_secret = '5iSOFztWXx5pc2nlzxzTrzwuZSEMXRf8l2sggosuS8VNo8rTeL'
access_token = '788065986502987777-aNqg4hWd3AqNGSo6H26D0pbzCFkMyfN'
access_token_secret = '4vtX80WsjFqPEyj25T6gqXZ0BJsLkpBOGLxcHLJuIuz0o'
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
# Remove pretrained model if it already exists.
!rm -r cardiffnlp/
# Test the Roberta Sentiment Analysis model
model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)
sentiment_task("Johnny Depp")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
rm: cannot remove 'cardiffnlp/': No such file or directory


[{'label': 'Neutral', 'score': 0.6338866353034973}]

In [ ]:
# SENTIMENT AND INTENT ANALYSIS
!rm -r cardiffnlp/
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

MODEL = f"cardiffnlp/twitter-xlm-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

# Test Sentiment Analysis Model on sample text below
text = "crap shit worse bad pollution negative horse cat"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)

# text = "Good night 😊"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)

# Print labels and scores
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
1) Negative 0.9479
2) Neutral 0.0417
3) Positive 0.0104


In [ ]:
# Python Script to Extract tweets of a
# particular Hashtag using Tweepy and Pandas

!rm -r cardiffnlp/
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoModelWithLMHead
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

tokenizer1 = AutoTokenizer.from_pretrained("cardiffnlp/twitter-xlm-roberta-base-sentiment")
config1 = AutoConfig.from_pretrained("cardiffnlp/twitter-xlm-roberta-base-sentiment")

# Sentiment Analysis Model
model1 = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-xlm-roberta-base-sentiment")
model1.save_pretrained("cardiffnlp/twitter-xlm-roberta-base-sentiment")

# Intent Analysis Model
tokenizer2 = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-e2m-intent")
model2 = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-e2m-intent")

def get_intent(event, max_length=16):
  input_text = "%s </s>" % event
  features = tokenizer2([input_text], return_tensors='pt')

  output = model2.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=max_length)

  return tokenizer2.decode(output[0])

# function to display data of each tweet
def printtweetdata(n, ith_tweet):
    print()
    print(f"Tweet {n}:")
    print(f"Username:{ith_tweet[0]}")
    print(f"Description:{ith_tweet[1]}")
    print(f"Location:{ith_tweet[2]}")
    print(f"Following Count:{ith_tweet[3]}")
    print(f"Follower Count:{ith_tweet[4]}")
    print(f"Total Tweets:{ith_tweet[5]}")
    print(f"Retweet Count:{ith_tweet[6]}")
    print(f"Tweet Text:{ith_tweet[7]}")
    print(f"Hashtags Used:{ith_tweet[8]}")
    print(f"Positive:{ith_tweet[9]}")
    print(f"Neutral:{ith_tweet[10]}")
    print(f"Negative:{ith_tweet[11]}")
    print(f"Intention:{ith_tweet[12]}")


# function to perform data extraction
def scrape(words, date_since, numtweet):

    # Creating DataFrame using pandas
    db = pd.DataFrame(columns=['username',
                'description',
                'location',
                'following',
                'followers',
                'totaltweets',
                'retweetcount',
                'text',
                'hashtags',
                'positive',
                'neutral',
                'negative',
                'intention'])

    # We are using .Cursor() to search
    # through twitter for the required tweets.
    # The number of tweets can be
    # restricted using .items(number of tweets)
    tweets = tweepy.Cursor(api.search_tweets,
              words, lang="en",
              since_id=date_since,
              tweet_mode='extended').items(numtweet)


    # .Cursor() returns an iterable object. Each item in
    # the iterator has various attributes
    # that you can access to
    # get information about each tweet
    list_tweets = [tweet for tweet in tweets]

    # Counter to maintain Tweet Count
    i = 1

    # we will iterate over each tweet in the
    # list for extracting information about each tweet
    for tweet in list_tweets:
        username = tweet.user.screen_name
        description = tweet.user.description
        location = tweet.user.location
        following = tweet.user.friends_count
        followers = tweet.user.followers_count
        totaltweets = tweet.user.statuses_count
        retweetcount = tweet.retweet_count
        hashtags = tweet.entities['hashtags']

        # Retweets can be distinguished by
        # a retweeted_status attribute,
        # in case it is an invalid reference,
        # except block will be executed
        try:
            text = tweet.retweeted_status.full_text
        except AttributeError:
            text = tweet.full_text
        hashtext = list()
        for j in range(0, len(hashtags)):
            hashtext.append(hashtags[j]['text'])
      
        # Sentiment Analysis
        preprocessed_text = preprocess(text)
        encoded_input = tokenizer1(preprocessed_text, return_tensors='pt')
        output = model1(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)

        # Print labels and scores
        ranking = np.argsort(scores)
        ranking = ranking[::-1]
        for j in range(scores.shape[0]):
            l = config.id2label[ranking[j]]
            s = scores[ranking[j]]

            if l == "Positive":
              positive = np.round(float(s), 4)
            elif l == "Neutral":
              neutral = np.round(float(s), 4)
            else:
              negative = np.round(float(s), 4)

        event = text
        intention = get_intent(event)  

        # Here we are appending all the
        # extracted information in the DataFrame
        ith_tweet = [username, description,
              location, following,
              followers, totaltweets,
              retweetcount, text, hashtext, positive, neutral, negative, intention]
        db.loc[len(db)] = ith_tweet

        # Function call to print tweet data on screen
        printtweetdata(i, ith_tweet)
        i = i + 1
    
    # we will save our database as a CSV file.
    filename = 'scraped_tweets.csv'
    db.to_csv(filename)

if __name__ == '__main__':

		# Enter Text or Hastag below and initial date
		words = "crisis"
		# yyyy-mm-dd
		date_since = "2022-04-01"

		# number of tweets you want to extract in one run
		numtweet = 5
		scrape(words, date_since, numtweet)
		print('Scraping has completed!')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.



Tweet 1:
Username:shaun_vids
Description:he/him.  
https://t.co/CXFt8dAqbV 
https://t.co/Z2XYWcMC95
Location:
Following Count:629
Follower Count:293614
Total Tweets:22644
Retweet Count:0
Tweet Text:in the neutral fiction world, a crisis is a crisis, interrupting the norm. in our world, crisis is an inherent, and necessary, result of capitalism. we're always building to the next bust. "crisis coming" reads "business as usual" to the ones causing the crisis
Hashtags Used:[]
Positive:0.0365
Neutral:0.1738
Negative:0.7898
Intention:<pad> to be a failure</s>

Tweet 2:
Username:K_Brown1996
Description:25👸🏾
Location:Cocoa, FL☀️
Following Count:242
Follower Count:503
Total Tweets:80478
Retweet Count:37393
Tweet Text:“Millennials aren’t having children”

The world: 
School shootings
Baby formula shortage 
Inflation 
$7.25 minimum wage
Housing crisis
Poor health care 
High mortality rate 
Student loan debt
War in Europe 
White supremacy 
Women’s rights in jeopardy
Hashtags Used:[]
Positive:0.01